In [1]:
import torch
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer, util

In [2]:
ca_ru = ET.parse('data/full data/t1989-9-canada-russian-federation-bit-1989.xml')
ca_cz = ET.parse('data/full data/t1990-14-canada-czech-republic-bit-1990.xml')


In [9]:
root_ru = ca_ru.getroot()
root_cz = ca_cz.getroot()
article1_ru = root_ru[1][2][0]
article1_cz = root_cz[1][2][0]

sentences_ru = []
sentences_cz = []

for child in article1_ru:
   sentences_ru.append(child.text)
for child in article1_cz:
   sentences_cz.append(child.text)

print(len(sentences_cz))


16


In [4]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

sentence_ru_embeddings = model.encode(sentences_ru)
sentence_cz_embeddings = model.encode(sentences_cz)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [18]:
for id, embd in enumerate(sentence_cz_embeddings):

    scores = util.cos_sim(embd, sentence_ru_embeddings)

    if max(scores[0])<0.9:
        print(sentences_cz[id])


In [67]:
def compare_all(doc1_path, doc2_path):
    doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
    root1, root2 = doc1.getroot(), doc2.getroot()

    num_pages = max(len(root1[1][2]), len(root2[1][2]))
    print('num_pages:', num_pages)
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')

    for page_id in range(num_pages):
        print('page_id:', page_id)
        all_the_same = True
        page1 = root1[1][2][page_id]
        page2 = root2[1][2][page_id]
        sentences_on_page1 = [child.text for child in page1]
        sentences_on_page2 = [child.text for child in page2]

        sentence_embeddings_on_page1 = model.encode(sentences_on_page1)
        sentence_embeddings_on_page2 = model.encode(sentences_on_page2)
        
        temp1 = []
        for id, embd in enumerate(sentence_embeddings_on_page1):
            scores = util.cos_sim(embd, sentence_embeddings_on_page2)
            if max(scores[0])<0.9:
                all_the_same = False
                temp1.append(sentences_on_page1[id])
        if temp1:
            print("The following sentences appear in doc1 but not in doc2:")
            for elem in temp1:
                print(elem)

        if not all_the_same:
            print('\n')

        temp2 = []
        for id, embd in enumerate(sentence_embeddings_on_page2):
            scores = util.cos_sim(embd, sentence_embeddings_on_page1)
            if max(scores[0])<0.9:
                all_the_same = False
                temp2.append(sentences_on_page2[id])
        if temp2:
            print("The following sentences appear in doc2 but not in doc1:")
            for elem in temp2:
                print(elem)
            

        if all_the_same:
            print("----[All the sentences in this page are the same.]----")
        print('\n\n')

compare_all('data/full data/t1989-9-canada-russian-federation-bit-1989.xml', 'data/full data/t1990-14-canada-czech-republic-bit-1990.xml')

num_pages: 14
page_id: 0
The following sentences appear in doc2 but not in doc1:
(d) The term "territory" means:
(ii) In respect of the Czech and Slovak Federal Republic, the territory of the Czech and Slovak Federal Republic.



page_id: 1
----[All the sentences in this page are the same.]----



page_id: 2
----[All the sentences in this page are the same.]----



page_id: 3
The following sentences appear in doc1 but not in doc2:
(d) Any existing or future convention relating to double taxation or other fiscal matters.


The following sentences appear in doc2 but not in doc1:
(d) Any existing or future convention relating to taxation.



page_id: 4
----[All the sentences in this page are the same.]----



page_id: 5
----[All the sentences in this page are the same.]----



page_id: 6
The following sentences appear in doc1 but not in doc2:
(a) The returns accruing from any investment;
In any convertible currency agreed upon between the investor and the Contracting Party concerned at th